Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Image Classification Using Scikit-learn

## Prerequisites

*    [ A Kubernetes cluster deployed on Azure Stack Hub, connected to Azure through ARC](https://github.com/Azure/AML-Kubernetes/blob/master/docs/ASH/AML-ARC-Compute.md).
     

*     [Datastore setup in Azure Machine Learning workspace backed up by Azure Stack Hub storage account](https://github.com/Azure/AML-Kubernetes/blob/master/docs/ASH/Train-AzureArc.md) 


*      Last but not least, you need to be able to run a Notebook. 

   If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the configuration Notebook located at [here](https://github.com/Azure/MachineLearningNotebooks) first. This sets you up with a working config file that has information on your workspace, subscription id, etc.

## Initialize AzureML workspace

Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`. 

If you haven't done already please go to `config.json` file and fill in your workspace information.

In [ ]:
from azureml.core.workspace import Workspace,  ComputeTarget
from azureml.exceptions import ComputeTargetException

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

## Download mnist data

Perform pip install azureml-opendatasets to get the open dataset package, use this function to download mnist data later. This allows you to avoid download the data again when you run this notebook multiple times. The actual download time may take 2 minutes.

In [ ]:
from azureml.core import Dataset
from azureml.opendatasets import MNIST
import os

def download_mnist_data():
    data_folder = os.path.join(os.getcwd(), 'mnist_data')
    os.makedirs(data_folder, exist_ok=True)

    mnist_file_dataset = MNIST.get_file_dataset()
    path = mnist_file_dataset.download(data_folder, overwrite=True)
    downloaded_folder = os.path.dirname(path[0])
    print("downloaded to", downloaded_folder)
    
    return downloaded_folder


## Prepare the dataset

The above download_mnist_data() function will download four files  t10k-images-idx3-ubyte.gz, t10k-labels-idx1-ubyte.gz, train-images-idx3-ubyte.gz and train-labels-idx1-ubyte.gz to downloaded_folder.  Your next step is to upload these files to datastore of the workspace, and then registered as dataset in the workspace. 

"datastore_name" is the name of the datastore you setup in [this step](https://github.com/Azure/AML-Kubernetes/blob/master/docs/ASH/Train-AzureArc.md).

Upload and dataset registration take less than 1 min.

In [ ]:
from azureml.core import Workspace, Dataset, Datastore

dataset_name = "mnist_ash_o"
datastore_name = "<NAME_OF_ASH_HOSTED_AML_DATASTORE>"

if dataset_name not  in ws.datasets:
    downloaded_folder = download_mnist_data()
    datastore =  Datastore.get(ws, datastore_name)
    
    src_dir, target_path =downloaded_folder, 'mnistdataash'
    datastore.upload(src_dir, target_path)

    # register data uploaded as AML dataset
    datastore_paths = [(datastore, target_path)]
    mnist_ds = Dataset.File.from_files(path=datastore_paths)
    mnist_ds.register(ws, dataset_name, "mnist data from http://yann.lecun.com/exdb/mnist/")

## Setup compute target

Find the attach name for the Arc enabled  Azure Stack Hub kubernetes cluster in your AzureML workspace to create a ComputeTarget:

attach_name is the attached name for your ASH cluster you setup in [this step](https://github.com/Azure/AML-Kubernetes/blob/master/docs/ASH/AML-ARC-Compute.md)

In [ ]:
from azureml.core.compute import KubernetesCompute

attach_name = "<NAME_OF_AML_ATTACHED_COMPUTE_OF_YOUR_ASH_CLUSTER>"
arcK_target = KubernetesCompute(ws, attach_name)

## Configure the training job and submit

### Create an experiement

In [ ]:
from azureml.core import Experiment

experiment_name = 'mnist-demo'

exp = Experiment(workspace=ws, name=experiment_name)

### Create an environment

In [ ]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# to install required packages
env = Environment('tutorial-env')
cd = CondaDependencies.create(pip_packages=['azureml-dataset-runtime[pandas,fuse]', 'azureml-defaults'], conda_packages = ['scikit-learn==0.22.1'])

env.python.conda_dependencies = cd

### Configure the training job

The training takes about 15 mins with vm size comparable  to Standard_DS3_v2

In [ ]:
from azureml.core import ScriptRunConfig

args = ['--data-folder', ws.datasets[dataset_name].as_mount(), '--regularization', 0.5]
script_folder =  "mnist_script"
src = ScriptRunConfig(source_directory=script_folder,
                      script='train.py', 
                      arguments=args,
                      compute_target=arcK_target,
                      environment=env)

### Submit the job

Run your experiment by submitting your ScriptRunConfig object. Note that this call is asynchronous.

In [ ]:
run = exp.submit(config=src)
run.wait_for_completion(show_output=True)  # specify True for a verbose log

### Register the model

Register the trained model.

In [ ]:
# register model
model_name='sklearn_mnist'
model = run.register_model(model_name=model_name,
                           model_path='outputs/sklearn_mnist_model.pkl')

The machine learning model named "sklearn_mnist" should be registered in your AzureML workspace.

## Test the registered model

To test the trained model, you can create (or use existing) a AKS cluster for serving the model using AML deployment.

Note: You will test the model in azure cloud because AzureML inferencing on Azure Stack Hub in currently unavailable.

In [ ]:
from azureml.core import Environment, Workspace, Model, ComputeTarget
from azureml.core.compute import AksCompute
from azureml.core.model import InferenceConfig
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.compute_target import ComputeTargetException
import numpy as np
import json

### Provision the AKS Cluster

This is a one time setup. You can reuse this cluster for multiple deployments after it has been created. If you delete the cluster or the resource group that contains it, then you would have to recreate it. It may take 5 mins to create a new AKS cluster.

In [ ]:
ws = Workspace.from_config()

# Choose a name for your AKS cluster
aks_name = 'aks-service-2'

# Verify that cluster does not exist already
try:
    aks_target = ComputeTarget(workspace=ws, name=aks_name)
    is_new_compute  = False
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # Use the default configuration (can also provide parameters to customize)
    prov_config = AksCompute.provisioning_configuration()

    # Create the cluster
    aks_target = ComputeTarget.create(workspace = ws, 
                                    name = aks_name, 
                                    provisioning_configuration = prov_config)
    is_new_compute  = True
    
print("using compute target: ", aks_target.name)

In [ ]:
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
model = Model(ws, model_name)

env = Environment('tutorial-env')
cd = CondaDependencies.create(pip_packages=['azureml-dataset-runtime[pandas,fuse]', 'azureml-defaults'], conda_packages = ['scikit-learn==0.22.1'])

env.python.conda_dependencies = cd

inference_config = InferenceConfig(entry_script="score.py", environment=env)
deploy_config = AksWebservice.deploy_configuration()

service = Model.deploy(workspace=ws, 
                       name='sklearn-mnist-svc', 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_target=aks_target,
                       deployment_config=deploy_config,
                       overwrite=True)

service.wait_for_deployment(show_output=True)

### Test with inputs

Here you may use the first image (saved as digit_7.jpg) from test set, as show here:
![fishy](digit_7.jpg)

In [ ]:
from mnist_script.utils import load_data
import os
import glob

data_folder = os.path.join(os.getcwd(), 'mnist_data')

X_test = load_data(glob.glob(os.path.join(data_folder,"**/t10k-images-idx3-ubyte.gz"), recursive=True)[0], False) / 255.0
y_test = load_data(glob.glob(os.path.join(data_folder,"**/t10k-labels-idx1-ubyte.gz"), recursive=True)[0], True).reshape(-1)

import json
test = json.dumps({"data": X_test.tolist()[:1]})
test = bytes(test, encoding='utf8')

# get predicted digits:

predictions = service.run(input_data=test)
print(predictions)

You should get 7 as predicted digit for the first image in test set .

### Delete the newly created cluster

Note: This is important if you wish to avoid the cost of this cluster

In [ ]:
if is_new_compute:
    aks_target.delete()
    service.delete()

## Next steps

1. Learn how to [distributed training with pytorch](../distributed-cifar10/distributed-pytorch-cifar10.ipynb)
2. Learn how to [distributed training with tensorflow](../distributed-cifar10/distributed-tf2-cifar10.ipynb)
3. Learn Pipeline Steps with [Object Segmentation](../object-segmentation-on-azure-stack/)